In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader,TensorDataset
import torch.utils.data as data
import numpy as np
import scipy.io as sio
import math
import argparse
import random
import os

In [2]:
BATCH_SIZE = 128
EPISODE = 500000
TEST_EPISODE = 1000
LEARNING_RATE =1e-5
Nom = 1e-5
Weight_Deacy =1e-5
GPU = 0
Margin = 1

In [3]:
print("init dataset")
##################################参数##################################################################
dataroot = '../data'
dataset = 'AwA1_data'
image_embedding = 'res101'               #ResNet101层
class_embedding = 'original_att'         #属性表达 85-d
#######################################读取视觉特征###################################################################

matcontent = sio.loadmat(dataroot + "/" + dataset + "/" + image_embedding + ".mat")  #scipy loadmat
 
feature = matcontent['features'].T         #转置 30478x2048 每一行是一个完整的样本

label = matcontent['labels'].astype(int).squeeze() - 1   #matlab begin 1 ,numpy begin 0
########################################读取属性特征###########################################################

matcontent = sio.loadmat(dataroot + "/" + dataset + "/" + class_embedding + "_splits.mat")

    
# numpy array index starts from 0, matlab starts from 1
trainval_loc = matcontent['trainval_loc'].squeeze() - 1    #squeeze()去掉维度中的1 AxBx1 --->AxB

test_seen_loc = matcontent['test_seen_loc'].squeeze() - 1
test_unseen_loc = matcontent['test_unseen_loc'].squeeze() - 1

attribute = matcontent['att'].T    #转置 50x85 每行是整个属性向量

x = feature[trainval_loc]                      # train_features trainval里面是图片的编号 begin with 0 ，19832个
train_label = label[trainval_loc].astype(int)  # train_label  int类型没变 每个图片的lable 19832个
train_id = np.unique(train_label)

att = attribute[train_label]                   # train attributes 每个图片的属性 19832个

########################add negative pairs#######################
x_test = feature[test_unseen_loc]                   # test_feature 5685个
test_label = label[test_unseen_loc].astype(int)     # test_label   5685个

x_test_seen = feature[test_seen_loc]                #test_seen_feature 4958个
test_label_seen = label[test_seen_loc].astype(int)  # test_seen_label  4958个
    
test_id = np.unique(test_label)                     # test_id  10个类 ，unique去重
att_pro = attribute[test_id]                        # test_attribute 每一类的属性向量 10x85

# train set
train_features = torch.from_numpy(x)   #np-->tensor
#train_fearures_negative = torch.from_numpy(x_negative)

sample_attributes=[]
train_label = torch.from_numpy(train_label).unsqueeze(1) #每张图片的属性转化 ，unsqueeze(1)就是插入到第一维度 AxB维-->Ax1xB
#train_negative_label = torch.from_numpy(x_negative_label).unsqueeze(1)
# attributes
all_attributes = np.array(attribute)  #所有50类属性转变为numpy数组???属性向量仍然用的numpy类型 没有转化为pytorch
#print(all_attributes)
    
#print('-'*50)
attributes = torch.from_numpy(attribute) 
#print(attribute)
# test set

test_features = torch.from_numpy(x_test)
#print(test_features.shape)

test_label = torch.from_numpy(test_label).unsqueeze(1)
#print(test_label.shape)

testclasses_id = np.array(test_id)
#print(testclasses_id.shape)

test_attributes = torch.from_numpy(att_pro).float()
#print(test_attributes.shape)

test_seen_features = torch.from_numpy(x_test_seen)
#print(test_seen_features.shape)

test_seen_label = torch.from_numpy(test_label_seen)

train_data = TensorDataset( train_label, train_features )
#train_data = TensorDataset(train_label, train_features, train_fearures_negative)

#################here need new code to make triplet data#####################
print('-'*100)

init dataset
----------------------------------------------------------------------------------------------------


In [4]:
from my_net_2 import AttributeNetwork
from my_net_2 import TripletNetwork_cos

In [5]:
# init network
print("init networks")
attribute_network = AttributeNetwork(85,1024,1024,2048)  #85d属性 1024隐藏层 2048输出 85d到2048d
triplet_network = TripletNetwork_cos(attribute_network)  #metric learning   
triplet_network.cuda(GPU)   
#attribute_network_optim = torch.optim.Adam(attribute_network.parameters(), lr=LEARNING_RATE, weight_decay=1e-5)
#优化器adam 学习率 正则1e-5

#attribute_network_scheduler = StepLR(attribute_network_optim, step_size=200000, gamma=0.5)
#学习率每200k步 乘0.5
#triplet_network_optim = torch.optim.Adam(triplet_network.parameters(), lr=LEARNING_RATE)
#
triplet_network_optim = torch.optim.Adam(triplet_network.parameters(), lr=LEARNING_RATE, 
                                         weight_decay=Weight_Deacy)
triplet_network_scheduler = StepLR(triplet_network_optim , step_size=200000 , gamma=0.5)
#
print('-'*100)

init networks
----------------------------------------------------------------------------------------------------


In [6]:
def compute_accuracy(test_features, test_label, test_id, test_attributes):
    
    test_data = TensorDataset(test_features, test_label)
    test_batch = 32
    test_loader = DataLoader(test_data, batch_size=test_batch, shuffle=False)
    total_rewards = 0

    sample_labels = test_id
    sample_attributes = test_attributes
    class_num = sample_attributes.shape[0]
    test_size = test_features.shape[0]

    print("class num:", class_num)

    for batch_features,batch_labels in test_loader:

        batch_size = batch_labels.shape[0]
        batch_features_ext = torch.from_numpy(batch_features.numpy().repeat(class_num, 0))
        batch_features_ext = Variable(batch_features_ext).cuda(GPU).float()  # 32*1024

        #print(batch_features_ext)

        sample_features = attribute_network(Variable(sample_attributes).cuda(GPU).float())
        sample_features_ext = sample_features.repeat(batch_size, 1)
        #print(sample_features_ext.shape)


        relations = F.cosine_similarity(batch_features_ext, sample_features_ext).view(-1, class_num)
        re_batch_labels = []
        for label in batch_labels.numpy():
            index = np.argwhere(sample_labels == label)
            re_batch_labels.append(index[0][0])
        re_batch_labels = torch.cuda.LongTensor(re_batch_labels)


        _, predict_labels = torch.min(relations.data, 1)
        #print(predict_labels)
        rewards = [1 if predict_labels[j] == re_batch_labels[j] else 0 for j in range(batch_size)]
        total_rewards += np.sum(rewards)
    test_accuracy = total_rewards/1.0/test_size
    return  test_accuracy

In [7]:
print("training...")
last_accuracy = 0.0
for episode in range(EPISODE):
    #attribute_network.train()
    triplet_network_scheduler.step(episode)

    train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)

    batch_labels, batch_features = train_loader.__iter__().next()
    
    batch_features_negative = np.empty_like(batch_features.numpy())
    
    batch_attributes = torch.Tensor([all_attributes[i] for i in batch_labels.numpy()]).squeeze(1)
    
    batch_features = Variable(batch_features).cuda(GPU).float()  # 32*2048
    
    for i in range(batch_features_negative.shape[0]):
        pick=np.random.choice(np.where(batch_labels.numpy()!=train_id)[0], replace=True)
        batch_features_negative[i] = batch_features[pick]    
    
    batch_features_negative = torch.from_numpy(batch_features_negative)
    batch_features_negative = Variable(batch_features_negative).cuda(GPU).float()  # 32*2048
    
    #batch_attributes = attribute_network(Variable(batch_attributes).cuda(GPU))
    batch_attributes = Variable(batch_attributes).cuda(GPU)
    
    dista, distb, embedded_x, embedded_y, embedded_z = triplet_network(batch_attributes, batch_features, batch_features_negative)
    
    criterion = nn.MarginRankingLoss(margin = Margin).cuda(GPU)
    
    target = torch.FloatTensor(dista.size()).fill_(-1).cuda(GPU)
    
    
    loss_triplet= criterion(dista, distb, target)
    loss_embedd = embedded_x.norm(2) + embedded_y.norm(2) + embedded_z.norm(2)
    loss = loss_triplet + Nom * loss_embedd

    # update
    triplet_network.zero_grad()
    loss.backward()
    triplet_network_optim.step()
    
    if (episode+1)%1000 == 0:
        print("episode:", episode+1, "loss", loss.data)
    if (episode+1)%1000 == 0:
        print("Testing...")
        #attribute_network.eval()
        zsl_accuracy = compute_accuracy(test_features, test_label, test_id, test_attributes)
        gzsl_unseen_accuracy = compute_accuracy(test_features, test_label, np.arange(50), attributes)
        gzsl_seen_accuracy = compute_accuracy(test_seen_features, test_seen_label, np.arange(50), attributes)
        H = 2 * gzsl_seen_accuracy * gzsl_unseen_accuracy / (gzsl_unseen_accuracy + gzsl_seen_accuracy)
        print('zsl:', zsl_accuracy)
        print('gzsl: unseen=%.4f , seen=%.4f , h=%.4f' % (gzsl_unseen_accuracy , gzsl_seen_accuracy, H))
        print('_'*100)

training...
episode: 1000 loss tensor(0.8946, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.46279683377308706
gzsl: unseen=0.0406 , seen=0.6658 , h=0.0766
____________________________________________________________________________________________________
episode: 2000 loss tensor(0.8505, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.4534740545294635
gzsl: unseen=0.0427 , seen=0.7055 , h=0.0806
____________________________________________________________________________________________________
episode: 3000 loss tensor(0.8384, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.4990325417766051
gzsl: unseen=0.0470 , seen=0.7196 , h=0.0882
____________________________________________________________________________________________________
episode: 4000 loss tensor(0.8080, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5343887423043095
gzsl: unseen=0.0529 , seen=0.7273 , h

episode: 31000 loss tensor(0.8099, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.58311345646438
gzsl: unseen=0.3646 , seen=0.7261 , h=0.4855
____________________________________________________________________________________________________
episode: 32000 loss tensor(0.7966, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5838170624450307
gzsl: unseen=0.3634 , seen=0.7289 , h=0.4850
____________________________________________________________________________________________________
episode: 33000 loss tensor(0.8200, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5854001759014952
gzsl: unseen=0.3733 , seen=0.7243 , h=0.4926
____________________________________________________________________________________________________
episode: 34000 loss tensor(0.8026, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5780123131046614
gzsl: unseen=0.3699 , seen=0.7211 , h=0.4890
___

episode: 61000 loss tensor(0.7863, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5489885664028145
gzsl: unseen=0.4417 , seen=0.6662 , h=0.5312
____________________________________________________________________________________________________
episode: 62000 loss tensor(0.7958, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5609498680738786
gzsl: unseen=0.4426 , seen=0.6694 , h=0.5329
____________________________________________________________________________________________________
episode: 63000 loss tensor(0.8106, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5461741424802111
gzsl: unseen=0.4338 , seen=0.6646 , h=0.5249
____________________________________________________________________________________________________
episode: 64000 loss tensor(0.8013, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.557080035180299
gzsl: unseen=0.4461 , seen=0.6632 , h=0.5334
__

episode: 91000 loss tensor(0.7932, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5340369393139842
gzsl: unseen=0.4617 , seen=0.6226 , h=0.5303
____________________________________________________________________________________________________
episode: 92000 loss tensor(0.7834, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5342128408091469
gzsl: unseen=0.4540 , seen=0.6299 , h=0.5277
____________________________________________________________________________________________________
episode: 93000 loss tensor(0.7983, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5333333333333333
gzsl: unseen=0.4524 , seen=0.6327 , h=0.5276
____________________________________________________________________________________________________
episode: 94000 loss tensor(0.7951, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5357959542656112
gzsl: unseen=0.4579 , seen=0.6327 , h=0.5313
_

episode: 121000 loss tensor(0.8020, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5410729991204926
gzsl: unseen=0.4769 , seen=0.6127 , h=0.5363
____________________________________________________________________________________________________
episode: 122000 loss tensor(0.7994, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5294635004397538
gzsl: unseen=0.4586 , seen=0.6150 , h=0.5254
____________________________________________________________________________________________________
episode: 123000 loss tensor(0.7909, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.544415127528584
gzsl: unseen=0.4904 , seen=0.5944 , h=0.5374
____________________________________________________________________________________________________
episode: 124000 loss tensor(0.7945, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5354441512752859
gzsl: unseen=0.4728 , seen=0.6111 , h=0.533

...

episode: 361000 loss tensor(0.7944, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5400175901495162
gzsl: unseen=0.4982 , seen=0.5623 , h=0.5283
____________________________________________________________________________________________________
episode: 362000 loss tensor(0.8044, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5227792436235708
gzsl: unseen=0.4827 , seen=0.5615 , h=0.5191
____________________________________________________________________________________________________
episode: 363000 loss tensor(0.7993, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5467018469656992
gzsl: unseen=0.5017 , seen=0.5704 , h=0.5338
____________________________________________________________________________________________________
episode: 364000 loss tensor(0.7929, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5414248021108179
gzsl: unseen=0.4960 , seen=0.5653 , h=0.52

episode: 391000 loss tensor(0.8004, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.545646437994723
gzsl: unseen=0.4999 , seen=0.5734 , h=0.5341
____________________________________________________________________________________________________
episode: 392000 loss tensor(0.7938, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5282321899736148
gzsl: unseen=0.4876 , seen=0.5581 , h=0.5205
____________________________________________________________________________________________________
episode: 393000 loss tensor(0.7980, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5291116974494283
gzsl: unseen=0.4865 , seen=0.5625 , h=0.5218
____________________________________________________________________________________________________
episode: 394000 loss tensor(0.8035, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5196130167106421
gzsl: unseen=0.4790 , seen=0.5541 , h=0.513

episode: 421000 loss tensor(0.8080, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5349164467897977
gzsl: unseen=0.4953 , seen=0.5617 , h=0.5264
____________________________________________________________________________________________________
episode: 422000 loss tensor(0.7926, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.527528583992964
gzsl: unseen=0.4876 , seen=0.5597 , h=0.5212
____________________________________________________________________________________________________
episode: 423000 loss tensor(0.7985, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5394898856640281
gzsl: unseen=0.4985 , seen=0.5621 , h=0.5284
____________________________________________________________________________________________________
episode: 424000 loss tensor(0.7928, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5377308707124011
gzsl: unseen=0.4974 , seen=0.5666 , h=0.529

episode: 451000 loss tensor(0.8056, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.535971855760774
gzsl: unseen=0.4953 , seen=0.5573 , h=0.5245
____________________________________________________________________________________________________
episode: 452000 loss tensor(0.8061, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5375549692172383
gzsl: unseen=0.4976 , seen=0.5597 , h=0.5268
____________________________________________________________________________________________________
episode: 453000 loss tensor(0.8080, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5372031662269129
gzsl: unseen=0.4943 , seen=0.5621 , h=0.5260
____________________________________________________________________________________________________
episode: 454000 loss tensor(0.8006, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5222515391380826
gzsl: unseen=0.4813 , seen=0.5617 , h=0.518

episode: 481000 loss tensor(0.7818, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5354441512752859
gzsl: unseen=0.4982 , seen=0.5549 , h=0.5250
____________________________________________________________________________________________________
episode: 482000 loss tensor(0.8018, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5340369393139842
gzsl: unseen=0.4966 , seen=0.5601 , h=0.5264
____________________________________________________________________________________________________
episode: 483000 loss tensor(0.7877, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5387862796833773
gzsl: unseen=0.4978 , seen=0.5639 , h=0.5288
____________________________________________________________________________________________________
episode: 484000 loss tensor(0.7865, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5319261213720317
gzsl: unseen=0.4927 , seen=0.5593 , h=0.52